In [1]:
import numpy as np
%load_ext autoreload
%autoreload 2
import dotenv
import os
import importlib
import pandas as pd
from datetime import datetime
import logging

#import naut_mt5 as n5
#from naut_mt5 import data_utils

import nautilus_trader
# test instrument provider
from nautilus_trader.test_kit.providers import TestInstrumentProvider
from nautilus_trader.persistence.wranglers import QuoteTickDataWrangler
from nautilus_trader.persistence.catalog import ParquetDataCatalog
import os
from nautilus_trader.data.engine import ParquetDataCatalog
from nautilus_trader.model.instruments import Instrument
from nautilus_trader.model.data import BarType, Bar
import pandas as pd
from pandas import DataFrame

from nautilus_trader.persistence.wranglers import BarDataWrangler
from nautilus_trader.model.identifiers import InstrumentId, Venue, Symbol
from nautilus_trader.model.data import BarType, BarSpecification, BarAggregation
from nautilus_trader.model.data import QuoteTick
from nautilus_trader.model.enums import AssetClass
from nautilus_trader.model.instruments import CurrencyPair
from nautilus_trader.model.instruments import Cfd
from nautilus_trader.model.objects import Currency
from nautilus_trader.model.objects import Money
from nautilus_trader.model.objects import Price
from nautilus_trader.model.objects import Quantity
from nautilus_trader.model.currencies import USD
from decimal import Decimal

In [68]:
catalog = ParquetDataCatalog("./catalog")
symbol = Symbol("XAU/USD")
venue = Venue("SIM")
instrument_id = InstrumentId(symbol, venue)

print(f"Catalog: {catalog}")
print(f"Symbol: {symbol}")
print(f"Venue: {venue}")
print(f"Instrument id: {instrument_id}")


symbol_str = symbol.value
base_currency = symbol_str[:3]
quote_currency = symbol_str[-3:]
price_precision = 2

instrument = CurrencyPair(
    instrument_id=instrument_id,
    raw_symbol=symbol,
    base_currency=Currency.from_str(base_currency),
    quote_currency=Currency.from_str(quote_currency),
    price_precision=price_precision,
    size_precision=0,
    price_increment=Price(1 / 10 ** price_precision, price_precision),
    size_increment=Quantity.from_int(1),
    lot_size=Quantity.from_str("100"),
    max_quantity=Quantity.from_str("1e7"),
    min_quantity=Quantity.from_str("1"),
    max_price=None,
    min_price=None,
    max_notional=Money(50_000_000.00, USD),
    min_notional=Money(1.00, USD), # todo: check this
    margin_init=Decimal("0.03"),
    margin_maint=Decimal("0.03"),
    maker_fee=Decimal("0.00002"),
    taker_fee=Decimal("0.00002"),
    ts_event=0,
    ts_init=0,
)

Catalog: <nautilus_trader.persistence.catalog.parquet.ParquetDataCatalog object at 0x13fc74fb0>
Symbol: XAU/USD
Venue: SIM
Instrument id: XAU/USD.SIM


In [69]:
catalog.list_data_types()
catalog.instruments()

[CurrencyPair(id=EURUSD.SIM_EIGHTCAP, raw_symbol=EURUSD, asset_class=FX, instrument_class=SPOT, quote_currency=USD, is_inverse=False, price_precision=5, price_increment=0.00001, size_precision=0, size_increment=1, multiplier=1, lot_size=1000, margin_init=0.03, margin_maint=0.03, maker_fee=0.00002, taker_fee=0.00002, info=None),
 CurrencyPair(id=XAUUSD.SIM, raw_symbol=XAUUSD, asset_class=FX, instrument_class=SPOT, quote_currency=USD, is_inverse=False, price_precision=2, price_increment=0.01, size_precision=0, size_increment=1, multiplier=1, lot_size=100, margin_init=0.03, margin_maint=0.03, maker_fee=0.00002, taker_fee=0.00002, info=None)]

In [70]:
csv_path = './XAUUSD_data_m1.csv'
#df = pd.read_csv(csv_path, index_col="time", parse_dates=True)
#df.index = pd.to_datetime(df.index, format="mixed")

df = pd.read_csv(csv_path)
print(df.head())
print(df.tail())

                  time     open     high      low    close  tick_volume  \
0  2024-11-29 05:05:00  2657.76  2658.25  2657.74  2658.06           56   
1  2024-11-29 05:06:00  2658.07  2658.07  2657.29  2657.56           76   
2  2024-11-29 05:07:00  2657.54  2657.94  2657.31  2657.31           76   
3  2024-11-29 05:08:00  2657.18  2657.40  2657.09  2657.10           70   
4  2024-11-29 05:09:00  2657.10  2657.10  2655.77  2655.77          116   

   spread  real_volume  
0      18            0  
1      18            0  
2      16            0  
3      18            0  
4      18            0  
                       time     open     high      low    close  tick_volume  \
999995  2022-02-07 03:20:00  1811.44  1811.50  1811.05  1811.14          145   
999996  2022-02-07 03:21:00  1811.10  1811.42  1810.99  1811.13          126   
999997  2022-02-07 03:22:00  1811.13  1811.37  1811.07  1811.11          104   
999998  2022-02-07 03:23:00  1811.12  1811.70  1811.10  1811.68          113   

In [71]:
df.time = pd.to_datetime(df.time)

df.time = df.time.astype('int')
df

,time,open,high,low,close,tick_volume,spread,real_volume
0,1732856700000000000,2657.76,2658.25,2657.74,2658.06,56,18,0
1,1732856760000000000,2658.07,2658.07,2657.29,2657.56,76,18,0
2,1732856820000000000,2657.54,2657.94,2657.31,2657.31,76,16,0
3,1732856880000000000,2657.18,2657.40,2657.09,2657.10,70,18,0
4,1732856940000000000,2657.10,2657.10,2655.77,2655.77,116,18,0
...,...,...,...,...,...,...,...,...
999995,1644204000000000000,1811.44,1811.50,1811.05,1811.14,145,14,0
999996,1644204060000000000,1811.10,1811.42,1810.99,1811.13,126,13,0
999997,1644204120000000000,1811.13,1811.37,1811.07,1811.11,104,19,0
999998,1644204180000000000,1811.12,1811.70,1811.10,1811.68,113,20,0


In [72]:
duplicates = df[df.duplicated(subset='time', keep=False)]
if not duplicates.empty:
    print("Duplicate rows found:")
    print(duplicates)
else:
    print("No duplicate rows found.")

No duplicate rows found.


In [73]:
if not df['time'].is_monotonic_increasing:
    print("The 'time' column is not sorted. First violation:")
    print(df[df['time'] != df['time'].sort_values().reset_index(drop=True)].head())
else:
    print("'time' column is correctly sorted in the CSV.")


The 'time' column is not sorted. First violation:
                  time     open     high      low    close  tick_volume  \
0  1732856700000000000  2657.76  2658.25  2657.74  2658.06           56   
1  1732856760000000000  2658.07  2658.07  2657.29  2657.56           76   
2  1732856820000000000  2657.54  2657.94  2657.31  2657.31           76   
3  1732856880000000000  2657.18  2657.40  2657.09  2657.10           70   
4  1732856940000000000  2657.10  2657.10  2655.77  2655.77          116   

   spread  real_volume  
0      18            0  
1      18            0  
2      16            0  
3      18            0  
4      18            0  


In [74]:
# Ensure time is sorted correctly as a numeric or datetime column
df['time'] = pd.to_datetime(df['time'])  # If it's not already datetime
df['time_ns'] = df['time'].astype('int64')  # Convert to nanoseconds for comparison

# Find where `time` decreases or is non-monotonic
violations = df['time_ns'].diff() < 0

# Get indices and rows where violations occur
violation_indices = df[violations].index
print(f"Number of violations: {len(violation_indices)}")

# Print the violations
if len(violation_indices) > 0:
    print("First violation:")
    print(df.loc[violation_indices[0] - 1:violation_indices[0] + 1])  # Show context
else:
    print("No violations found.")


Number of violations: 999
First violation:
                    time     open     high      low    close  tick_volume  \
999  2024-11-29 21:44:00  2650.02  2650.33  2650.02  2650.30           60   
1000 2024-11-28 08:55:00  2636.08  2636.44  2635.94  2636.10          112   
1001 2024-11-28 08:56:00  2636.17  2636.55  2636.09  2636.47           83   

      spread  real_volume              time_ns  
999       31            0  1732916640000000000  
1000      17            0  1732784100000000000  
1001      18            0  1732784160000000000  


In [75]:
violations = sum(1 for i in range(1, len(bars)) if bars[i].ts_init < bars[i - 1].ts_init)
print(f"Total out-of-order bars: {violations}")


Total out-of-order bars: 0


In [76]:
# Ensure 'time' is in datetime format for proper sorting
df['time'] = pd.to_datetime(df['time'])

# Sort the DataFrame by the 'time' column
df = df.sort_values(by='time', ascending=True)

# Remove duplicates based on the 'time' column, keeping the first occurrence
df = df.drop_duplicates(subset='time', keep='first')

# Reset the index after sorting and deduplication
df.set_index('time', inplace=True)


print(f"Data sorted and deduplicated. Number of rows: {len(df)}")


Data sorted and deduplicated. Number of rows: 1000000


In [77]:
if not df.index.is_monotonic_increasing:
    print("'time' column is NOT correctly sorted in the CSV.")

In [85]:
bar_type = BarType.from_str(f"{instrument_id.value}-1-MINUTE-MID-EXTERNAL")
print(f"Bar type: {bar_type}")

Bar type: XAU/USD.SIM-1-MINUTE-MID-EXTERNAL


In [86]:
wrangler = BarDataWrangler(bar_type, instrument)
bars: list[Bar] = wrangler.process(df)
#basename_template = "part-{i}" + f"-from-{int(start.timestamp())}-to-{int(end.timestamp())}"

In [87]:
print("bars: ", len(bars))

bars:  1000000


In [88]:
catalog.write_data([instrument],)

In [89]:
catalog.write_data(bars,)

In [90]:
catalog.instruments()

[CurrencyPair(id=EURUSD.SIM_EIGHTCAP, raw_symbol=EURUSD, asset_class=FX, instrument_class=SPOT, quote_currency=USD, is_inverse=False, price_precision=5, price_increment=0.00001, size_precision=0, size_increment=1, multiplier=1, lot_size=1000, margin_init=0.03, margin_maint=0.03, maker_fee=0.00002, taker_fee=0.00002, info=None),
 CurrencyPair(id=XAU/USD.SIM, raw_symbol=XAU/USD, asset_class=FX, instrument_class=SPOT, quote_currency=USD, is_inverse=False, price_precision=2, price_increment=0.01, size_precision=0, size_increment=1, multiplier=1, lot_size=100, margin_init=0.03, margin_maint=0.03, maker_fee=0.00002, taker_fee=0.00002, info=None)]

In [97]:
catalog.bars(instrument_ids=[instrument_id])

[Bar(XAU/USD.SIM-1-MINUTE-MID-EXTERNAL,1807.02,1807.55,1806.96,1807.44,138,1643967720000000000),
 Bar(XAU/USD.SIM-1-MINUTE-MID-EXTERNAL,1807.44,1807.46,1807.21,1807.26,44,1643967780000000000),
 Bar(XAU/USD.SIM-1-MINUTE-MID-EXTERNAL,1807.32,1807.32,1806.65,1806.65,52,1643967840000000000),
 Bar(XAU/USD.SIM-1-MINUTE-MID-EXTERNAL,1806.68,1807.13,1806.52,1806.57,126,1643967900000000000),
 Bar(XAU/USD.SIM-1-MINUTE-MID-EXTERNAL,1806.63,1806.76,1806.50,1806.72,98,1643967960000000000),
 Bar(XAU/USD.SIM-1-MINUTE-MID-EXTERNAL,1806.69,1807.29,1806.69,1807.23,103,1643968020000000000),
 Bar(XAU/USD.SIM-1-MINUTE-MID-EXTERNAL,1807.26,1807.26,1806.88,1807.02,107,1643968080000000000),
 Bar(XAU/USD.SIM-1-MINUTE-MID-EXTERNAL,1806.99,1807.11,1806.73,1806.88,139,1643968140000000000),
 Bar(XAU/USD.SIM-1-MINUTE-MID-EXTERNAL,1806.87,1806.93,1806.72,1806.72,105,1643968200000000000),
 Bar(XAU/USD.SIM-1-MINUTE-MID-EXTERNAL,1806.65,1807.05,1806.62,1806.94,60,1643968260000000000),
 Bar(XAU/USD.SIM-1-MINUTE-MID-EXTE